In [7]:
# Install required packages (if not installed)
!pip install chromadb pandas sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 92.6 MB/s eta 0:0

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, f1_score
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
from transformers import AutoModelForSeq2SeqLM, pipeline

In [3]:
# === Imports ===
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification

# ===============================
# 1. SETUP DATA & BASELINE MODELS
# ===============================

# Load CSV Data for RAG
filepath = '/content/pace_graduate_programs (1).csv'  # Replace with your file path
df = pd.read_csv(filepath)
program_names = df['Program Name'].tolist()
program_links = df['Program Link'].tolist()

# Embed Program Names
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(program_names)

# Initialize Chroma Vector Store
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = chroma_client.get_or_create_collection(name="pace_programs")

# Add data if collection is empty
if collection.count() == 0:
    collection.add(
        documents=program_names,
        metadatas=[{"link": link} for link in program_links],
        embeddings=embeddings.tolist(),
        ids=[f"id{i}" for i in range(len(program_names))]
    )

# Semantic Search
def search_programs(user_query, top_k=5):
    query_embed = embedding_model.encode([user_query])[0].tolist()
    results = collection.query(query_embeddings=[query_embed], n_results=top_k)
    programs = results['documents'][0]
    metadata = results['metadatas'][0]
    return list(zip(programs, [meta['link'] for meta in metadata]))

# Baseline Flan-T5 Model
baseline_model_name = "google/flan-t5-small"
baseline_tokenizer = AutoTokenizer.from_pretrained(baseline_model_name)
baseline_model = AutoModelForSeq2SeqLM.from_pretrained(baseline_model_name)
baseline_pipeline = pipeline("text2text-generation", model=baseline_model, tokenizer=baseline_tokenizer)

# ===============================
# 2. SETUP SHONA SLANG CLASSIFIER
# ===============================

# shona_model_path = "/content/shona_chatbot_model"  # Your fine-tuned model path
shona_model_repo_id = "HappymoreMasoka/shona-intent-classification-model" # Hugging Face repo ID

shona_tokenizer = AutoTokenizer.from_pretrained(shona_model_repo_id, trust_remote_code=True)
shona_model = AutoModelForSequenceClassification.from_pretrained(shona_model_repo_id, trust_remote_code=True)
shona_classifier = pipeline("text-classification", model=shona_model, tokenizer=shona_tokenizer, return_all_scores=True)

# Load label map
unique_intents = sorted(pd.read_csv('/content/slang_dataset_with_contexts_and_intent.csv')['intent'].unique())
label_map = {intent: i for i, intent in enumerate(unique_intents)}
id2label = {v: k for k, v in label_map.items()}

# ===============================
# 3. CLASSIFICATION FUNCTIONS
# ===============================

def classify_intent(text):
    result = shona_classifier(text)
    pred = max(result[0], key=lambda x: x["score"])
    label_index = int(pred["label"].replace("LABEL_", ""))
    intent_name = id2label[label_index]
    return intent_name, pred["score"]

# ===============================
# 4. RULE-BASED HANDLER FOR SHONA BOT
# ===============================
def handle_intent_response(intent, user_input):
    intent = intent.lower()
    user_input_lower = user_input.lower()

    # -------------------
    # GREETINGS
    # -------------------
    if intent == "greeting":
        greetings = [
            "Hesi shamwari! Uri sei hako?",
            "Mhoroi, zvakanaka here?",
            "Mangwanani akanaka!",
            "Masikati akanaka, shamwari."
        ]
        return greetings[0]  # could randomize

    elif intent == "religious_greeting":
        return "Mangwanani akanaka hama dzaMwari 🙏🏽. Mwari ngavakuropafadzei."

    elif intent == "farewell":
        return "Sarai zvakanaka! Tichataurirana zvakare."

    # -------------------
    # APPRECIATION / GRATITUDE
    # -------------------
    elif intent == "gratitude":
        return "Ndokutendai zvikuru! Makaita basa."

    elif intent == "celebration":
        return "Makorokoto! Mwari akuropafadzei pamufaro uyu 🎉"

    # -------------------
    # REQUESTS / HELP
    # -------------------
    elif intent == "request":
        return "Ndiri kunzwa chikumbiro chako. Chii chaunoda kubatsirwa nacho?"

    # -------------------
    # RELIGION
    # -------------------
    elif intent == "prayer_or_blessing":
        return "Tinosimudzira minamato yedu kuna Mwari. Garai makasimba 🙏🏽"

    elif intent == "religion":
        return "Bata Department reReligion pa: www.religion@pace.edu"

    # -------------------
    # FAMILY / FRIENDSHIP
    # -------------------
    elif intent == "family":
        return "Mhuri inokosha! pa Pace University tinoda uzive kut you are part of the bigger family?"

    elif intent == "friendship":
        return "Zvirikufamba sei shamwari yangu?"

    elif intent == "romantic":
        return "Aaaah, zviri pachena kuti rudo ruripo ❤️"

    # -------------------
    # EDUCATION / WORK
    # -------------------
    elif intent == "educational":
        return "Zvekudzidza hazviperere pano. Unoda ruzivo pamusoro pemapurogiramu api?"

    elif intent == "work":
        return "bata carear department pa www.carears@pace.edu"

    # -------------------
    # APPLICATION FLOW
    # -------------------
    elif any(word in user_input_lower for word in ["apply", "nyora", "kunyora", "register", "application","program"]):
        name = input("ShonaBot: Ndokumbirawo zita rako rizere: ")
        education = input("ShonaBot: Wakadzidza kupi kare? (e.g., BSc, diploma, etc.): ")
        email = input("ShonaBot: Email yako ndeipi?: ")
        print(f"\n[INFO] Application received:\nName: {name}\nEducation: {education}\nEmail: {email}")
        return "Waita apply! Chikumbiro chako chakatumirwa kuchikoro. Tichakutumira zvimwe ruzivo 💼"

    # -------------------
    # MISC / FALLBACK
    # -------------------
    elif intent == "sympathy":
        return "Ndinonzwa nemi. Mwari vakupai nyaradzo munguva ino yakaoma."

    elif intent == "health_update":
        return "Ndapota chengeta hutano hwako. Kana zviri serious enda kuchipatara."

    elif intent == "sports":
        return "yes we have a lot of sporting activities pa Pace University, bata sports department pa sports@pace.edu"

    elif intent == "humor":
        return "Wandiseka zve! 😂😂😂😂"

    # Default fallback
    return "Ndanzwisisa zvawataura, asi ndinoda rumwe ruzivo kuti ndikubatsire."



# ===============================
# 5. RAG + FLAN-T5 GENERATION
# ===============================

def generate_answer_with_rag(query):
    intent, confidence = classify_intent(query)
    programs = search_programs(query)
    context = "\n".join([f"{name} – {link}" for name, link in programs])
    prompt = f"""You are a helpful assistant that answers questions about graduate programs at Pace University.
The user's intent is: {intent} (confidence: {confidence:.2f}).

Programs:
{context}

Question: {query}

Answer:"""
    answer = baseline_pipeline(prompt, max_new_tokens=150)[0]['generated_text']
    return intent, confidence, answer

# ===============================
# 6. EXIT CHECK
# ===============================

def is_exit(text):
    return text.strip().lower() in ["exit", "quit", "bye"]

# ===============================
# 7. INTERACTIVE CHAT LOOP
# ===============================

print("💬 Welcome! Type your question in Shona slang or English. (Type 'exit' to quit)\n")

while True:
    user_input = input("Iwe: ")

    if is_exit(user_input):
        print("PaceBot: Zvakanaka, tichaonana zvakare! 🎓")
        break

    # Shona chatbot
    intent, confidence = classify_intent(user_input)
    special_response = handle_intent_response(intent, user_input)

    print(f"\n📌 Detected Intent: {intent} (confidence {confidence:.2f})")

    if special_response is not None:
        print(f"🤖 ShonaBot: {special_response}")
    else:
        print("🤖 ShonaBot: Ndine urombo, handina mhinduro yakajeka pane izvozvo.")

    # Baseline RAG
    intent_rag, conf_rag, rag_answer = generate_answer_with_rag(user_input)
    print(f"🤖 Baseline RAG: {rag_answer}")
    print("-" * 100)

Device set to use cpu
Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


💬 Welcome! Type your question in Shona slang or English. (Type 'exit' to quit)

Iwe: madii

📌 Detected Intent: Greeting (confidence 1.00)
🤖 ShonaBot: Hesi shamwari! Uri sei hako?
🤖 Baseline RAG: Pace University
----------------------------------------------------------------------------------------------------
Iwe: mune ma program api pa pace
ShonaBot: Ndokumbirawo zita rako rizere: chibaba chacho
ShonaBot: Wakadzidza kupi kare? (e.g., BSc, diploma, etc.): telecomz
ShonaBot: Email yako ndeipi?: hmasoka@gmail.com

[INFO] Application received:
Name: chibaba chacho
Education: telecomz
Email: hmasoka@gmail.com

📌 Detected Intent: general (confidence 1.00)
🤖 ShonaBot: Waita apply! Chikumbiro chako chakatumirwa kuchikoro. Tichakutumira zvimwe ruzivo 💼
🤖 Baseline RAG: Pace University-Lenox Hill Hospital Physician Assistant Program, MS – https://www.pace.edu/program/pace-university-lenox-hill-hospital-physician-assistant-program-ms Higher Education Administration and Student Affairs, MA – http